In [130]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import re

def penn_to_wn(tag):
  #  return tag
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('V'):
        return 'v'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    return None

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    #print("wn_tag", wn_tag)
    if wn_tag is None:
        #print("exting", word, tag, wn_tag)
        return None
    try:
        #print(word, tag, wn_tag)
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 


In [159]:
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    #print(sentence1)
    sentence2 = pos_tag(word_tokenize(sentence2))
    #print(sentence2)
    #print("tagged words",[tagged_word for tagged_word in sentence1])
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    #print("synset:",synsets1)
    synsets1 = [ss for ss in synsets1 if ss]
    #print(synsets1)
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0.001

    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        # print [synset.path_similarity(ss) for ss in synsets2]
        try:
            best_score = max([synset.path_similarity(ss) for ss in synsets2])
        except:
            best_score = 0.0
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1

    # Average the values
    score /= count
    return score

In [160]:
def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    #print sentence1,sentence2
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2

In [205]:
def sentence_similarity_query(sentence1):
    with open("coffee_shop_faq.txt") as f:
        faq_split = f.read().split("?")
        question = ""
        question_answer_list = []
        for faq in faq_split:
            faq_list = faq.split("\n")
            #print('list',faq_list)
            answer = " ".join(faq_list[:-1])
            #print("ANSWER:",answer)
            if(len(question)>10 and len(answer)>10):
                question = " ".join(re.compile('\w+').findall(question))
                #print(question)
                answer = " ".join(re.compile('\w+').findall(answer))
                #print(answer)
                question_answer_list.append([question,answer])
            question = faq_list[-1]+"?"
        sentence1  = " ".join(re.compile('\w+').findall(sentence1))
        score_list = []
        for sets in question_answer_list:
            score_list.append(symmetric_sentence_similarity(sentence1,sets[0]))
        question_query = question_answer_list[score_list.index(max(score_list))][0]
        answer_query = question_answer_list[score_list.index(max(score_list))][1]
        print(question_query)
        print(answer_query)
        return answer_query

sentence_similarity_query("What payment methods do you accept")

Do you accept exchanges
At this time we are unable to process exchanges If you wish to exchange an item please contact our offices within 1 week from the day your shipment arrives For more detailed information about our return and exchange policies please view our span style color blue font weight bold a href http www marthastewartcafe com terms of sale target _blank Terms of Sale a span


'At this time we are unable to process exchanges If you wish to exchange an item please contact our offices within 1 week from the day your shipment arrives For more detailed information about our return and exchange policies please view our span style color blue font weight bold a href http www marthastewartcafe com terms of sale target _blank Terms of Sale a span'

In [201]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /anaconda3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [218]:
"What payment methods do you accept"

wn.synsets('exchanges')

[Synset('exchange.n.01'),
 Synset('exchange.n.02'),
 Synset('exchange.n.03'),
 Synset('exchange.n.04'),
 Synset('central.n.01'),
 Synset('exchange.n.06'),
 Synset('rally.n.05'),
 Synset('exchange.n.08'),
 Synset('substitution.n.02'),
 Synset('exchange.n.10'),
 Synset('exchange.n.11'),
 Synset('exchange.v.01'),
 Synset('change.v.06'),
 Synset('switch_over.v.01'),
 Synset('exchange.v.04'),
 Synset('substitute.v.01'),
 Synset('commute.v.04')]

In [198]:
sentence1 = "Can i get a 9 - refund"
sentence1  = " ".join(re.compile('\w+').findall(sentence1))

In [199]:
sentence1

'Can i get a 9 refund'